# Hierarchical Dirichlet Process Topic Model Implementation on SLO Twitter Dataset

### Joseph Jinn and Keith VanderLinden

### Import libraries and set parameters:

We import the requisite libraries, custom utility functions, and set the parameters for our various imported libraries.

In [ ]:
# Import libraries.
import logging as log
import warnings
import time
import pandas as pd
import numpy as np
from gensim import corpora
from sklearn.feature_extraction.text import CountVectorizer
from matplotlib import pyplot as plt
import seaborn as sns

# Import custom utility functions.
import topic_extraction_utility_functions as lda_util

#############################################################

# Pandas options.
pd.options.display.max_rows = None
pd.options.display.max_columns = None
pd.options.display.width = None
pd.options.display.max_colwidth = 1000
# Pandas float precision display.
pd.set_option('precision', 12)
# Seaborn setting.
sns.set()
# Don't output these types of warnings to terminal.
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
# Matplotlib log settings.
mylog = log.getLogger("matplotlib")
mylog.setLevel(log.INFO)

"""
Turn debug log statements for various sections of code on/off.
(adjust log level as necessary)
"""
log.basicConfig(level=log.INFO)
log.disable(level=log.DEBUG)

### Pre-process and Post-process Tweets:

We tokenize our Tweets using our dataset preprocessing and postprocessing function.  Please refer to "topic_extraction_utility_functions.py" for the full code-base and comments on what is done to the Tweet text and user description text.

In [ ]:
# Test on our topic modeling dataset.
tweet_dataset_preprocessor(
    "D:/Dropbox/summer-research-2019/jupyter-notebooks/attribute-datasets/"
    "twitter-dataset-7-10-19-test-subset-100-examples.csv",
    "D:/Dropbox/summer-research-2019/jupyter-notebooks/attribute-datasets/"
    "twitter-dataset-7-10-19-lda-ready-tweet-text-with-hashtags-excluded-created-7-17-19.csv",
    "text_derived")

# Test on our topic modeling dataset.
tweet_dataset_preprocessor(
    "D:/Dropbox/summer-research-2019/jupyter-notebooks/attribute-datasets/"
    "twitter-dataset-7-10-19-test-subset-100-examples.csv",
    "D:/Dropbox/summer-research-2019/jupyter-notebooks/attribute-datasets/"
    "twitter-dataset-7-10-19-lda-ready-user-description-text-with-hashtags-excluded-created-7-17-19.csv",
    "user_description")

### Import and prepare the preprocessed dataset for use in HDP topic extraction:

We follow the general format of insertion into a Pandas dataframe, isolating the column of interest, and generating a dictionary of words and corpus of documents.  Please refer to the code comments for details on the specific steps for the entire process.

In [ ]:
# # Import the dataset (relative path).
# tweet_dataset_processed = \
#     pd.read_csv("twitter-dataset-7-10-19-lda-ready-tweet-text-with-hashtags-excluded-created-7-17-19.csv", sep=",")

# Import the dataset (absolute path).
tweet_dataset_processed = \
    pd.read_csv("D:/Dropbox/summer-research-2019/jupyter-notebooks/attribute-datasets/"
                "twitter-dataset-7-10-19-lda-ready-tweet-text-with-hashtags-excluded-created-7-17-19.csv", sep=",")

# # Import the dataset (test/debug).
# tweet_dataset_processed = \
#     pd.read_csv("twitter-dataset-7-10-19-lda-ready-tweet-text-test.csv", sep=",")

# # Import the dataset (test/debug).
# tweet_dataset_processed = \
#     pd.read_csv("D:/Dropbox/summer-research-2019/jupyter-notebooks/attribute-datasets/"
#                 "twitter-dataset-7-10-19-lda-ready-tweet-text-test.csv", sep=",")

# Reindex and shuffle the data randomly.
tweet_dataset_processed = tweet_dataset_processed.reindex(
    pd.np.random.permutation(tweet_dataset_processed.index))

# Generate a Pandas dataframe.
tweet_text_dataframe = pd.DataFrame(tweet_dataset_processed)

# # Print shape and column names.
# log.info(f"\nThe shape of the Tweet text dataframe:")
# log.info(f"{tweet_text_dataframe.shape}\n")
# log.info(f"\nThe columns of the Tweet text dataframe:")
# log.info(f"{tweet_text_dataframe.columns}\n")

# Print shape and column names.
log.info("\nThe shape of the Tweet text dataframe:")
log.info(tweet_text_dataframe.shape)
log.info("\nThe columns of the Tweet text dataframe:")
log.info(tweet_text_dataframe.columns)

# Drop any NaN or empty Tweet rows in dataframe (or else CountVectorizer will blow up).
tweet_text_dataframe = tweet_text_dataframe.dropna()

# # Print shape and column names.
# log.info(f"\nThe shape of the Tweet text dataframe with NaN (empty) rows dropped:")
# log.info(f"{tweet_text_dataframe.shape}\n")
# log.info(f"\nThe columns of the Tweet text dataframe with NaN (empty) rows dropped:")
# log.info(f"{tweet_text_dataframe.columns}\n")

# Print shape and column names.
log.info("\nThe shape of the Tweet text dataframe with NaN (empty) rows dropped:")
log.info(tweet_text_dataframe.shape)
log.info("\nThe columns of the Tweet text dataframe with NaN (empty) rows dropped:")
log.info(tweet_text_dataframe.columns)

# Reindex everything.
tweet_text_dataframe.index = pd.RangeIndex(len(tweet_text_dataframe.index))

# Assign column names.
tweet_text_dataframe_column_names = ['text_derived', 'text_derived_preprocessed', 'text_derived_postprocessed']

# Rename column in dataframe.
tweet_text_dataframe.columns = tweet_text_dataframe_column_names

# Create input feature.
selected_features = tweet_text_dataframe[['text_derived_postprocessed']]
processed_features = selected_features.copy()

# # Check what we are using as inputs.
# log.info(f"\nA sample Tweet in our input feature:")
# log.info(f"{processed_features['text_derived_postprocessed'][0]}\n")

# Check what we are using as inputs.
log.info("\nA sample Tweet in our input feature:")
log.info(processed_features['text_derived_postprocessed'][0])

# Create feature set.
slo_feature_series = processed_features['text_derived_postprocessed']
slo_feature_series = pd.Series(slo_feature_series)
slo_feature_list = slo_feature_series.tolist()

# Convert feature list of sentences to comma-separated dictionary of words.
words = [[text for text in tweet.split()] for tweet in slo_feature_list]
# log.info(f"\nDictionary of individual words:")
# log.info(f"{words[0]}\n")
log.info("\nDictionary of individual words:")
log.info(words[0])

# # Create the Gensim dictionary of words.
# dictionary = corpora.Dictionary(words)
# log.info(f"\nGensim dictionary of tokenized words.")
# log.info(f"{dictionary}\n")
# log.info(f"\nGensim dictionary of tokenized words with index ID's.")
# log.info(f"{dictionary.token2id}\n")

# Create the Gensim dictionary of words.
dictionary = corpora.Dictionary(words)
log.info("\nGensim dictionary of tokenized words.")
log.info(dictionary)
log.info("\n")
log.info("\nGensim dictionary of tokenized words with index ID's.")
log.info(dictionary.token2id)
log.info("\n")

# # Create the Gensim corpus of document term frequencies.
# corpus = [dictionary.doc2bow(word, allow_update=True) for word in words]
# log.info(f"# of documents in corpus: {len(corpus)}\n")
# log.info(f"\nSample of Gensim corpus of document-term frequencies.")
# log.info(f"{corpus[0:10]}\n")

# Create the Gensim corpus of document term frequencies.
corpus = [dictionary.doc2bow(word, allow_update=True) for word in words]
log.info("# of documents in corpus: " + str(len(corpus)) + "\n")
log.info("\nSample of Gensim corpus of document-term frequencies.")
log.info(corpus[0:10])
log.info("\n")

### Perform the topic extraction:

This function contains the code specific to each topic modeling library we utilize.  

In [ ]:
def hierarchical_dirichlet_process_topic_extraction():
    """
    Function performs topic extraction on Tweets using the Gensim HDP model.

    :return: None.
    """
    from gensim.test.utils import common_corpus, common_dictionary
    from gensim.models import HdpModel
    from gensim.sklearn_api import HdpTransformer

    # LDA can only use raw term counts for LDA because it is a probabilistic graphical model.
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
    tf = tf_vectorizer.fit_transform(slo_feature_series)
    tf_feature_names = tf_vectorizer.get_feature_names()

    # log.info("\n.fit_transform - Learn the vocabulary dictionary and return term-document matrix.")
    # log.info(f"{tf}\n")
    # log.info("\n.get_feature_names - Array mapping from feature integer indices to feature name")
    # log.info(f"{tf_feature_names}\n")

    log.info("\n.fit_transform - Learn the vocabulary dictionary and return term-document matrix.")
    log.info(tf)
    log.info("\n.get_feature_names - Array mapping from feature integer indices to feature name")
    log.info(tf_feature_names)

    # # Sample dictionary and corpus.
    # log.info(f"\nExample dictionary format for Gensim:")
    # log.info(f"{common_dictionary}\n")
    # log.info(f"\nExample corpus format for Gensim:")
    # log.info(f"{common_corpus}\n")

    # Train the HDP model.
    hdp = HdpModel(corpus, dictionary)
    time.sleep(3)

    # # For use as wrapper with Scikit-Learn API.
    # model = HdpTransformer(id2word=dictionary)
    # distribution = model.fit_transform(corpus)

    # Display the top words for each topic.
    topic_info = hdp.print_topics(num_topics=20, num_words=10)

    for topic in topic_info:
        print(topic)

Here, we call the topic modeling function and train it on our Twitter dataset.  We record the time it takes to process the entire dataset and extract topics.

In [ ]:
"""
Main function.  Execute the program.
"""
if __name__ == '__main__':
    my_start_time = time.time()
    ################################################
    """
    Perform the topic extraction.
    """
    hierarchical_dirichlet_process_topic_extraction()

    ################################################
    my_end_time = time.time()

    time_elapsed_in_seconds = (my_end_time - my_start_time)
    time_elapsed_in_minutes = (my_end_time - my_start_time) / 60.0
    time_elapsed_in_hours = (my_end_time - my_start_time) / 60.0 / 60.0
    # print(f"Time taken to process dataset: {time_elapsed_in_seconds} seconds, "
    #       f"{time_elapsed_in_minutes} minutes, {time_elapsed_in_hours} hours.")
    print("\n\nTime taken to process dataset: " + str(time_elapsed_in_seconds) + " seconds, " +
          str(time_elapsed_in_minutes) + " minutes, " + str(time_elapsed_in_hours) + " hours.\n")

### Topic Extraction Results on Twitter Dataset Tweet Text:

First run.  Output shows the weight of each of the top words associated with each topic.

In [ ]:
"""
(run 1)
(0, "0.044*adani + 0.022*coal + 0.008*santos + 0.007*job + 0.007*'s + 0.006*bhp + 0.006*project + 0.006*stop + 0.005*australia + 0.005*want")
(1, '0.076*í + 0.048*° + 0.042*tax + 0.040*½í² + 0.022*¼í¶\x93 + 0.021*adani + 0.016*pay + 0.013*$ + 0.011*coal + 0.010*energy')
(2, '0.144*$ + 0.027*bhp + 0.013*rio + 0.012*adani + 0.008*cba + 0.008*anz + 0.007*wbc + 0.007*nab + 0.006*coal + 0.006*price')
(3, "0.028*adani + 0.013*coal + 0.009*santos + 0.007*bhp + 0.006*rio + 0.005*australia + 0.005*tinto + 0.004*'s + 0.004*job + 0.004*$")
(4, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*job + 0.004*'s + 0.004*australia + 0.004*rio + 0.004*$ + 0.004*stop")
(5, "0.029*adani + 0.014*coal + 0.009*santos + 0.007*bhp + 0.004*australia + 0.004*'s + 0.004*job + 0.004*rio + 0.004*project + 0.004*$")
(6, "0.026*adani + 0.013*coal + 0.011*bhp + 0.007*santos + 0.007*rio + 0.006*tinto + 0.004*$ + 0.004*australia + 0.004*job + 0.004*'s")
(7, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*'s + 0.004*job + 0.004*australia + 0.004*stop + 0.004*rio + 0.004*$")
(8, "0.028*adani + 0.014*coal + 0.008*santos + 0.008*bhp + 0.004*rio + 0.004*'s + 0.004*australia + 0.004*job + 0.004*$ + 0.004*project")
(9, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*'s + 0.004*job + 0.004*australia + 0.004*rio + 0.004*$ + 0.004*stop")
(10, "0.029*adani + 0.014*coal + 0.008*santos + 0.008*bhp + 0.004*$ + 0.004*'s + 0.004*rio + 0.004*australia + 0.004*job + 0.004*project")
(11, "0.029*adani + 0.014*coal + 0.009*santos + 0.007*bhp + 0.004*'s + 0.004*job + 0.004*australia + 0.004*rio + 0.004*stop + 0.004*project")
(12, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*'s + 0.004*job + 0.004*australia + 0.004*rio + 0.004*$ + 0.004*project")
(13, "0.029*adani + 0.014*coal + 0.008*santos + 0.008*bhp + 0.004*rio + 0.004*'s + 0.004*australia + 0.004*job + 0.004*$ + 0.004*project")
(14, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*'s + 0.004*australia + 0.004*job + 0.004*rio + 0.004*$ + 0.004*project")
(15, "0.028*adani + 0.014*coal + 0.008*santos + 0.008*bhp + 0.007*$ + 0.005*í + 0.004*rio + 0.004*australia + 0.004*job + 0.004*'s")
(16, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*$ + 0.004*'s + 0.004*job + 0.004*australia + 0.004*rio + 0.004*project")
(17, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*'s + 0.004*job + 0.004*australia + 0.004*rio + 0.004*$ + 0.004*project")
(18, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*'s + 0.004*job + 0.004*australia + 0.004*rio + 0.004*$ + 0.004*stop")
(19, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.005*'s + 0.004*job + 0.004*australia + 0.004*rio + 0.004*$ + 0.004*project")


Time taken to process dataset: 1077.2826988697052 seconds, 17.95471164782842 minutes, 0.2992451941304737 hours.


Process finished with exit code 0

"""

Second run.  Output shows the weight of each of the top words associated with each topic.

In [ ]:
(run 2)
(0, "0.046*adani + 0.022*coal + 0.008*job + 0.007*bhp + 0.007*'s + 0.006*project + 0.006*stop + 0.006*santos + 0.005*australia + 0.005*want")
(1, '0.079*í + 0.048*° + 0.040*tax + 0.040*½í² + 0.022*¼í¶\x93 + 0.021*adani + 0.016*pay + 0.013*$ + 0.011*coal + 0.009*energy')
(2, '0.030*santos + 0.019*adani + 0.016*gas + 0.013*coal + 0.012*nsw + 0.011*water + 0.008*pilliga + 0.007*great + 0.006*project + 0.006*narrabri')
(3, '0.154*$ + 0.024*bhp + 0.014*adani + 0.010*rio + 0.008*cba + 0.008*wbc + 0.008*anz + 0.008*nab + 0.007*coal + 0.005*price')
(4, "0.030*adani + 0.017*coal + 0.008*santos + 0.007*bhp + 0.005*'s + 0.004*australia + 0.004*rio + 0.004*job + 0.004*project + 0.003*want")
(5, "0.029*adani + 0.016*coal + 0.010*santos + 0.007*bhp + 0.005*water + 0.005*gas + 0.004*'s + 0.004*point + 0.004*australia + 0.004*rio")
(6, "0.027*adani + 0.013*coal + 0.010*santos + 0.007*bhp + 0.006*'s + 0.004*project + 0.004*australia + 0.004*rio + 0.004*job + 0.004*$")
(7, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*'s + 0.004*job + 0.004*australia + 0.004*rio + 0.004*$ + 0.004*stop")
(8, "0.026*adani + 0.013*coal + 0.010*bhp + 0.009*rio + 0.007*santos + 0.007*tinto + 0.005*australia + 0.004*$ + 0.004*'s + 0.004*job")
(9, "0.028*adani + 0.015*coal + 0.008*santos + 0.007*bhp + 0.004*job + 0.004*'s + 0.004*australia + 0.004*$ + 0.004*rio + 0.004*project")
(10, "0.029*adani + 0.013*coal + 0.008*santos + 0.007*bhp + 0.005*rio + 0.005*want + 0.004*'s + 0.004*job + 0.004*australia + 0.004*$")
(11, "0.028*adani + 0.014*coal + 0.009*santos + 0.007*bhp + 0.004*'s + 0.004*australia + 0.004*job + 0.004*$ + 0.004*rio + 0.004*stop")
(12, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*job + 0.004*$ + 0.004*'s + 0.004*australia + 0.004*rio + 0.004*project")
(13, "0.030*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*'s + 0.004*australia + 0.004*job + 0.004*rio + 0.004*$ + 0.004*stop")
(14, "0.029*adani + 0.014*coal + 0.008*santos + 0.008*bhp + 0.004*job + 0.004*australia + 0.004*'s + 0.004*rio + 0.004*$ + 0.004*stop")
(15, "0.028*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*'s + 0.004*australia + 0.004*job + 0.004*rio + 0.004*$ + 0.003*stop")
(16, "0.029*adani + 0.014*coal + 0.008*santos + 0.008*bhp + 0.004*australia + 0.004*'s + 0.004*job + 0.004*$ + 0.004*rio + 0.004*project")
(17, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*'s + 0.004*australia + 0.004*job + 0.004*rio + 0.004*$ + 0.004*project")
(18, "0.029*adani + 0.014*coal + 0.009*santos + 0.007*bhp + 0.004*'s + 0.004*job + 0.004*australia + 0.004*rio + 0.004*$ + 0.004*stop")
(19, "0.029*adani + 0.014*coal + 0.008*santos + 0.007*bhp + 0.004*'s + 0.004*job + 0.004*australia + 0.004*rio + 0.004*$ + 0.004*project")


Time taken to process dataset: 1071.8396093845367 seconds, 17.863993489742278 minutes, 0.29773322482903797 hours.


Process finished with exit code 0

The results are sub-par using just default hyperparameter values.

## Resources Used:

- https://radimrehurek.com/gensim/models/hdpmodel.html
    - Gensim HDP topic modeling Class.
    